In [1]:
import os
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.{ext}")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.{ext}")

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False)
# group by imagetyp, camera and exptime
frames_table = frames_table.group_by(["mjd", "imagetyp", "spec", "camera", "expnum", "exptime", "neon", "hgne", "xenon", "argon", "ldls"])
# define groups and unique categories in those groups
frames_groups = frames_table.groups
unique_groups = frames_groups.keys

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
mjds = [60011]
cameras = ["r", "z"]
spec = 1
for mjd, camera in it.product(mjds, cameras):
    arcs = frames_groups[(unique_groups["mjd"]==mjd)&(unique_groups["imagetyp"]=="arc")&(unique_groups["spec"]=="sp1")]
    arcs.pprint(max_width=200)
    
    # let's pre-process the all arc frames
    parc_paths = []
    for arc_frame in arcs:
        arc_path = arc_frame["path"]
        arc_camera, arc_expnum = arc_frame["camera"], arc_frame["expnum"]

        parc_path = out_main_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits")

        imageMethod.preprocRawFrame_drp(
            in_image=arc_path,
            out_image=parc_path,
            subtract_overscan=True, unit="electron", compute_error=True,
            os_bound_y="1,4080", os_bound_x="2044,2077"
        )
        parc_paths.append(parc_path)

    imageMethod.createMasterFrame_drp(
        in_images=parc_paths,
        out_image=out_calib_path.format(mjd=mjd, kind="marc", camera=arc_camera, exptime=int(arc_frame["exptime"]), ext="fits")
    )

imagetyp spec  mjd  camera  expnum  ...  neon  ldls  hgne xenon                                                                   path                                                                 
-------- ---- ----- ------ -------- ... ----- ----- ----- ----- ---------------------------------------------------------------------------------------------------------------------------------------
     arc  sp1 60011     b1 00000230 ... False False False False /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/data/lvm/lab/60011/sdR-s-b1-00000230.fits.gz
     arc  sp1 60011     b1 00000231 ... False False False False /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/data/lvm/lab/60011/sdR-s-b1-00000231.fits.gz
     arc  sp1 60011     b1 00000232 ... False  True False False /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/data/lvm/lab/60011/sdR-s-b1-00000232.fits.gz


KeyboardInterrupt: 